# Scraping collegedata.com with Qt Webkit


In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

import requests
import time
from bs4 import BeautifulSoup
import re
import TIdatabase as ti

In [12]:
college_ids=['Princeton', 'Harvard', 'Yale', 'Columbia', 'Stanford', 'UChicago', 'MIT', 'Duke', 'UPenn', 'CalTech', 'JohnsHopkins', 'Dartmouth', 'Northwestern', 'Brown', 'Cornell', 'Vanderbilt', 'WashU', 'Rice', 'NotreDame', 'UCB', 'Emory', 'Georgetown', 'CarnegieMellon', 'UCLA', 'USC']
college_urls=[111, 444, 244, 399, 781, 327, 186, 1026, 67, 706, 1509, 403, 1803, 163, 787, 1562, 1720, 731, 1774, 1090, 1039, 1182, 204, 1093, 1138]
college_id_dict=dict(zip(college_ids,college_urls))
uni_list=['Princeton University', 'Harvard College', 'Yale University', 'Columbia University', 'Stanford University', 'University of Chicago', 'Massachusetts Institute of Technology', 'Duke University', 'University of Pennsylvania', 'California Institute of Technology', 'Johns Hopkins University', 'Dartmouth College', 'Northwestern University', 'Brown University', 'Cornell University', 'Vanderbilt University', 'Washington University in St. Louis', 'Rice University', 'University of Notre Dame', 'University of California, Berkeley', 'Emory University', 'Georgetown University', 'Carnegie Mellon University', 'University of California, Los Angeles', 'University of Southern California']
uni_name_dict = dict(zip(uni_list, college_ids))
uni_state=['NJ', 'MA', 'CT', 'NY', 'CA', 'IL', 'MA', 'NC', 'PA', 'CA', 'MD', 'NH', 'IL', 'RI', 'NY', 'TN', 'MO', 'TX', 'IN', 'CA', 'GA', 'DC', 'PA', 'CA', 'CA']
states_dict={'Alabama': 'AL',
 'Alaska': 'AK',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'National': 'NA',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Northern Mariana Islands': 'MP',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virgin Islands': 'VI',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY',
  'Other': 'Other'}
uni_state_dict = dict(zip(uni_list,uni_state))
baseurl='http://www.collegedata.com/cs/admissions/'
tracker_url='admissions_tracker_result.jhtml?schoolId='
student_url='admissions_profile_view.jhtml?profileName='

In [3]:
## Source: https://webscraping.com/blog/Scraping-multiple-JavaScript-webpages-with-webkit/
import sys
from PyQt4.QtCore import *
from PyQt4.QtGui import *
from PyQt4.QtWebKit import *

class Render(QWebPage):  
    def __init__(self, urls):
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.urls = urls 
        self.profileList=set()
        self.crawl()  
        self.app.exec_()  
      
    def crawl(self):  
        if self.urls:  
            url = self.urls.pop(0)    
            self.mainFrame().load(QUrl(url))  
        else:  
            self.app.quit()  
        
    def _loadFinished(self, result): 
        frame = self.mainFrame()  
        url = str(frame.url().toString())  
        html = frame.toHtml()  
        self.scrape(url, html)
        self.crawl()
    
    def scrape(self,url, html):
        soup = BeautifulSoup(str(html.toAscii()),'html.parser')     
        profiles=soup.find_all("a",href=re.compile(r"enterProfileByName"))
        for p in profiles:
            self.profileList.add(p.get("href").split("'")[1])

Execute the next cell to try out for 1 university and 2 years

In [4]:
#urls=[]
#schoolid=111
#schoolurl=baseurl+tracker_url+str(schoolid)+'&classYear='
#for year in range(2009,2011):
#    urls.append(schoolurl+str(year))
#r=Render(urls)
#print len(r.profileList)

Execute the next cell to try out for all universities and years 2009-2019. This takes about 55 minutes.

In [5]:
%%time
urls=[]
for school in college_id_dict:
    schoolurl=baseurl+tracker_url+str(college_id_dict[school])+'&classYear='
    for year in range(2009,2020):
        urls.append(schoolurl+str(year))
r=Render(urls)
print 'We found ', len(r.profileList), 'students'

We found  5010 students
CPU times: user 18min 8s, sys: 3min 9s, total: 21min 17s
Wall time: 40min 59s


In [63]:
# these are bad profiles without any information
r.profileList.remove('orangecat')
r.profileList.remove('j7Wa4')
print 'We found ', len(r.profileList), 'students'

We found  5007 students


In [59]:
genderdict = {'Male': -1, 'Female': 1}
highschooldict = {'Public': -1, 'Private': 1, 'Parochial': 1, 'Home-Schooled': 1}
minoritylist = ['african', 'hispanic', 'latin','indian', 'native', 'black', 'mexican','puerto','alaska','hawai','pacific island']
sats=[1600, 1560, 1510, 1460, 1420, 1380, 1340, 1300, 1260, 1220, 1190, 1150, 1110, 1070, 1030, 990, 950, 910, 870, 
      830, 790, 740, 690, 640, 590, 530]
act2satdict=dict(zip(range(11,37),sats))
booleandict={'': -1, 'X': 1}
statusdict={'Will Attend': 1, 'Accepted': 1, 'Applied': 0, 'Deferred': -1, 'Denied': -1, 'Not Applied': 0, 'Wait-Listed': -1, 'Withdrawn': 0, 'Pending': -1}

In [60]:
def getFromDict(dictionary,text):
    if dictionary.has_key(text):
        return dictionary[text]
    else:
        return 0
def isMinority(text):
    for m in minoritylist:
        if m in text:
            return 1
    return -1
def getScores(scores,fun):
    scores = [s.get_text().strip() for s in scores]
    while '' in scores:
        scores.remove('')
    if len(scores)>0:
        scores=[int(s) for s in scores]
        return fun(scores), len(scores)
    else:
        return None, 0
def getAdmissionTestScore(doc):
    satCRM,dummy = getScores(doc[0:2],sum)
    satW,dummy = getScores([doc[2]],np.max)
    act,dummy = getScores([doc[4]],np.max)
    if act == None:
        return (satCRM+satW)
    elif satCRM==None:
        if satW==None:
            print "Warning: no SAT writing score"
        return act2satdict[act]+satW
    else:
        return (max(act2satdict[act],satCRM)+satW)
def getCanAfford(text):
    if 'Yes' in text:
        return -1
    elif 'No' in text:
        return 1
    else:
        return 0
def removePunct(text):
    text = re.sub(r'([^\s\w]|_)+'," ",text)
    return re.sub('\s+',' ',text).encode('latin-1')

In [8]:
columns_student = ['classrank', 'admissionstest','AP','averageAP','SATsubject', 'GPA', 'GPA_w', 'program','schooltype',
            'intendedgradyear', 'addInfo', 'canAfford', 'female', 'MinorityGender','MinorityRace','international',
           'firstinfamily','sports','artist', 'workexp']
columns_uni = ['collegeID','earlyAppl','visited','alumni', 'outofstate', 'acceptStatus','acceptProb']

In [17]:
def getColumnValues(soup):
    values=dict(zip(columns_student,[None for i in range(len(columns_student))]))
    applications=[]
    # We start with the general information box at the top which includes class year, gender and ethnicity
    doc= soup.find("div",{"class": "general"})
    values['intendedgradyear'] = int(re.findall(r'\d{4}',doc.find("h1").get_text().split('Class of')[-1])[0]) # CLASS YEAR
    doc = doc.find_all("span")
    values['female']= getFromDict(genderdict,doc[0].get_text().strip()) # GENDER
    values['MinorityGender']= 1 if values['female']==0 else -1
    values['MinorityRace'] = isMinority(doc[1].get_text().strip().lower()) # MINORITY RACE
    values['program'] = removePunct(doc[2].get_text().strip()) # PROGRAM
    # Now we look at the academics box which includes GPA and high school info
    doc = soup.find("div", {"class": "academicswrap"}).find_all("span")
    values['schooltype']=getFromDict(highschooldict,doc[0].get_text().strip()) # SCHOOL TYPE
    state=getFromDict(states_dict,doc[1].get_text().strip()) # STATE OF STUDENT
    values['international'] = 1 if state=='Other' else -1
    values['GPA'] = float(doc[3].get_text()) # unweighted GPA
    values['GPA_w'] = float(doc[4].get_text()) if doc[4].get_text().strip()!='' else None # Weighted GPA
    # Next, we go to the test score box which includes SAT, ACT and AP info
    values['admissionstest'] = getAdmissionTestScore(soup.find("div", {"class": "testscorewrap"}).find_all("td")) #Admissions test
    values['SATsubject'] = len(soup.find("caption",text="SAT Subject Test Scores").next_sibling.next_sibling.find_all("tr")) # SAT SUBJECT
    ap_num = len(soup.find("caption",text="AP Examinations").next_sibling.next_sibling.find_all("tr"))
    values['AP']=ap_num #AP
    if ap_num>0:
        doc = soup.find("caption",text="AP Examinations").next_sibling.next_sibling.find_all("td")
        values['averageAP'],values['AP']= getScores(doc,np.mean) # AVERAGE AP
    doc = soup.find_all("div", {"class": "word"})
    doc = [d.get_text().strip() for d in doc]  
    values['addInfo']= removePunct(doc[0]+doc[1]+doc[2]) # Additional info
    # Next: the colleges applied to and the admission results for the admissions table
    doc = soup.find("table", {"class": "collchoice"})
    collegelist = doc.find("tbody").find_all("tr")
    for c in collegelist:
        uni = c.find("th").find("span").get_text().strip()
        if uni in uni_list:
            unirow = dict(zip(columns_uni,[None for i in range(6)]))
            unirow['collegeID']=uni_name_dict[uni]
            doc=c.find_all("td", {"class": "center"})
            unirow['earlyAppl']=booleandict[doc[0].get_text().strip()] # Early Admission
            unirow['alumni']=booleandict[doc[1].get_text().strip()] # Alumni/Legacy
            if values['sports']==None or values['sports']==0: 
                values['sports']=booleandict[doc[2].get_text().strip()] # Athlete
            doc = doc[2].next_sibling.find_next("span")
            unirow['acceptStatus']=getFromDict(statusdict,doc.get_text().strip()) # Admission status
            if values['canAfford']==None or values['canAfford']==0:
                values['canAfford']=getCanAfford(doc.find_next("span").get_text().strip())
            unirow['outofstate']= -1 if state==uni_state_dict[uni] else 1
            applications.append(unirow)
    return values, applications

In [64]:
%%time
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applForm' in locals()): del applForm
students=ti.Student()
colleges = ti.College()
applForm = ti.ApplForm()
for p in r.profileList:
    #print p
    profile_url=baseurl+student_url+p
    soup=BeautifulSoup(requests.get(profile_url).text,'html.parser')
    if soup.find("div", {"class": "academicswrap"})==None:
        print p, ' not Found'
        continue
    newrow, applications = getColumnValues(soup)
    studentID=students.insert(newrow)
    for app in applications:
        app['studentID']=studentID[0]
    applForm.insert(applications)
students.df.head()

CPU times: user 15min 14s, sys: 1min 12s, total: 16min 26s
Wall time: 1h 13min 5s


In [65]:
students.df.shape

(5007, 21)

In [66]:
applForm.df.head()

,studentID,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,PWY05BUB4I,Rice,-1,None,-1,-1,1,NaN
0,9FYGU63IH6,UPenn,-1,None,-1,1,-1,NaN
1,9FYGU63IH6,Princeton,-1,None,-1,1,-1,NaN
2,9FYGU63IH6,Harvard,1,None,-1,1,-1,NaN
3,9FYGU63IH6,Stanford,-1,None,-1,-1,-1,NaN


In [67]:
applForm.df.shape

(16062, 8)

In [68]:
students.save('collegedata_students.csv')
applForm.save('collegedata_applications.csv')

In [69]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applForm' in locals()): del applForm
students=ti.Student()
applForm = ti.ApplForm()
students.read('collegedata_students.csv')
applForm.read('collegedata_applications.csv')
students.df.head()

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,schooltype,intendedgradyear,addInfo,canAfford,female,MinorityGender,MinorityRace,international,firstinfamily,sports,artist,workexp
0,PWY05BUB4I,NaN,2290,7,5.000000,3,3.80,4.34,Biomedical engineering,-1,2017,Basketball outside of school violin cancer awa...,0,1,-1,-1,-1,NaN,-1,NaN,NaN
0,9FYGU63IH6,NaN,2070,5,4.400000,4,3.90,4.22,Mechanical Engineering,1,2018,Swimming 3 years Water Polo 3 years Foreign La...,1,1,-1,-1,-1,NaN,-1,NaN,NaN
0,BA3AWCCUQ5,NaN,2200,6,4.833333,3,4.00,4.58,Pre Med,-1,2012,City Government Youth Partnership Advisory Com...,0,-1,-1,-1,-1,NaN,-1,NaN,NaN
0,EDBIPXPZ4N,NaN,2210,4,4.250000,0,3.72,NaN,NaN,-1,2014,NaN,0,1,-1,-1,-1,NaN,-1,NaN,NaN
0,W9S7RTA8L8,NaN,2040,6,4.000000,0,3.88,NaN,English,-1,2018,NaN,1,-1,-1,-1,-1,NaN,-1,NaN,NaN


In [70]:
applForm.df.head()

,studentID,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,PWY05BUB4I,Rice,-1,NaN,-1,-1,1,NaN
0,9FYGU63IH6,UPenn,-1,NaN,-1,1,-1,NaN
1,9FYGU63IH6,Princeton,-1,NaN,-1,1,-1,NaN
2,9FYGU63IH6,Harvard,1,NaN,-1,1,-1,NaN
3,9FYGU63IH6,Stanford,-1,NaN,-1,-1,-1,NaN
